In [ ]:
# Google Drive bağlantısını kur ve veri seti klasörünü kullanıcıdan al
from google.colab import drive
drive.mount('/content/drive')

# Kullanıcıdan veri klasörünün adını girmesini iste (Örn: MyDrive/Dataset)
data_dir = input("Lütfen veri setinin bulunduğu Drive klasör yolunu girin (Örn: MyDrive/YSA): ")


Google Drive Bağlantısı ve Gerekli Kütüphanelerin Yüklenmesi

In [ ]:
# Gerekli kütüphaneleri içe aktaralım
import os
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Veri dengesizliği olması durumunda over-sampling için
try:
    from imblearn.over_sampling import RandomOverSampler
except ImportError:
    !pip install imbalanced-learn
    from imblearn.over_sampling import RandomOverSampler


Veri Yükleme ve Görüntüleme

In [ ]:
# Veri setlerini yükleyelim (dinamik yol)
try:
    train_metadata = pd.read_csv(f'/content/drive/{data_dir}/Metadata_Train.csv')
    test_metadata = pd.read_csv(f'/content/drive/{data_dir}/Metadata_Test.csv')
    
    print("Eğitim veri seti ilk 5 satır:")
    print(train_metadata.head())
    print("\nTest veri seti ilk 5 satır:")
    print(test_metadata.head())

    # Veri seti hakkında genel bilgiler
    print("\nEğitim veri seti bilgileri:")
    print(train_metadata.info())
    print("\nEksik veriler:")
    print(train_metadata.isnull().sum())
except FileNotFoundError:
    print("Hata: Dosya bulunamadı, lütfen dosya yolunu kontrol edin.")


Ses Dosyalarını Yükleme

In [ ]:
import os
from tqdm import tqdm
import librosa

def load_audio_files_in_batches(base_path, batch_size=50):
    files = os.listdir(base_path)
    error_files = []
    for i in range(0, len(files), batch_size):
        batch_files = files[i:i+batch_size]
        audio_data, sample_rates = [], []
        for file in tqdm(batch_files, desc=f"Batch {i//batch_size + 1}"):
            file_path = os.path.join(base_path, file)
            try:
                audio, sr = librosa.load(file_path, sr=22050)
                audio_data.append(audio)
                sample_rates.append(sr)
            except Exception as e:
                error_files.append((file, str(e)))
        yield audio_data, sample_rates, batch_files, error_files


In [ ]:
def load_audio_files(base_path):
    files = [f for f in os.listdir(base_path) if f.endswith(('.wav', '.mp3', '.flac'))]  # Yalnızca ses dosyalarını seç
    audio_data, sample_rates, error_files = [], [], []
    for file in tqdm(files, desc="Ses Dosyaları Yükleniyor"):
        file_path = os.path.join(base_path, file)
        try:
            audio, sr = librosa.load(file_path, sr=22050)
            audio_data.append(audio)
            sample_rates.append(sr)
        except Exception as e:
            error_files.append((file, str(e)))
    return audio_data, sample_rates, files, error_files



In [ ]:
import numpy as np

def load_audio_files(base_path):
    files = os.listdir(base_path)
    audio_data, sample_rates, error_files = [], [], []
    for file in tqdm(files, desc="Ses Dosyaları Yükleniyor"):
        file_path = os.path.join(base_path, file)
        try:
            if file.endswith('.npy'):  # NumPy dosyası için özel işlem
                audio = np.load(file_path)
                sr = 22050  # NumPy dosyasının örnekleme oranı bilinmiyorsa varsayılan kullanılır
            else:  # Ses dosyaları için Librosa
                audio, sr = librosa.load(file_path, sr=22050)
            audio_data.append(audio)
            sample_rates.append(sr)
        except Exception as e:
            error_files.append((file, str(e)))
    return audio_data, sample_rates, files, error_files


In [ ]:
def load_audio_files(base_path):
    files = [f for f in os.listdir(base_path) if f.endswith(('.wav', '.mp3', '.flac'))]  # Yalnızca ses dosyalarını seç
    audio_data, sample_rates, error_files = [], [], []
    for file in tqdm(files, desc="Ses Dosyaları Yükleniyor"):
        file_path = os.path.join(base_path, file)
        try:
            audio, sr = librosa.load(file_path, sr=22050)
            audio_data.append(audio)
            sample_rates.append(sr)
        except Exception as e:
            error_files.append((file, str(e)))
    return audio_data, sample_rates, files, error_files

train_base_path = '/content/drive/MyDrive/YSA/Train_submission/Train_submission'
all_audio_data, all_sample_rates, files, error_files = load_audio_files(train_base_path)

if error_files:
    print("\nYüklenemeyen dosyalar:")
    for file, error in error_files:
        print(f"Dosya: {file}, Hata: {error}")
else:
    print("\nTüm dosyalar başarıyla yüklendi!")


Özellik Çıkarma ve Etiketleme

In [ ]:
def extract_features(audio, sr):
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    return np.mean(mfccs.T, axis=0)

def extract_features_and_labels(audio_data, sample_rates, files):
    features, labels = [], []
    for idx, (audio, sr) in enumerate(zip(audio_data, sample_rates)):
        try:
            features.append(extract_features(audio, sr))
            if "Guitar" in files[idx]:
              labels.append("Guitar")
            elif "Drum" in files[idx]:
                labels.append("Drum")
            elif "Violin" in files[idx]:
                labels.append("Violin")
            elif "Piano" in files[idx]:
                labels.append("Piano")
            else:
                labels.append("Unknown")
        except Exception as e:
            print(f"Özellik çıkarma sırasında hata: {files[idx]}, Hata: {e}")
    return pd.DataFrame(features), labels

def extract_features(audio, sr):
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
    spectral_contrast = librosa.feature.spectral_contrast(y=audio, sr=sr)
    features = np.concatenate([
        np.mean(mfccs.T, axis=0),
        np.mean(chroma.T, axis=0),
        np.mean(spectral_contrast.T, axis=0)
    ])
    return features


features_df, labels = extract_features_and_labels(all_audio_data, all_sample_rates, files)
features_df['Label'] = labels
print(f"Toplam veri: {len(features_df)}")


In [ ]:
# Spektrogram görselleştirme fonksiyonu
def plot_spectrogram(audio, sr):
    plt.figure(figsize=(10, 4))
    spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr)
    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)
    librosa.display.specshow(spectrogram_db, sr=sr, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel-Spectrogram')
    plt.tight_layout()
    plt.show()

# Spektrogram görselleştirme
plot_spectrogram(all_audio_data[0], all_sample_rates[0])


Veri Artırma

In [ ]:
def add_noise(audio, noise_level=0.005):
    noise = np.random.normal(0, noise_level, audio.shape)
    return audio + noise

def shift_audio(audio, shift_max=0.2):
    shift = int(np.random.uniform(-shift_max, shift_max) * len(audio))
    return np.roll(audio, shift)

def change_speed_scipy(audio, speed_factor):
    try:
        new_length = int(len(audio) / speed_factor)
        return librosa.resample(audio, orig_sr=len(audio), target_sr=new_length)
    except Exception as e:
        return audio

def augment_audio_data(audio_data, labels):
    augmented_data, augmented_labels = [], []
    for audio, label in zip(audio_data, labels):
        augmented_data.extend([
            audio,
            add_noise(audio),
            shift_audio(audio),
            change_speed_scipy(audio, 1.2),
            change_speed_scipy(audio, 0.8)
        ])
        augmented_labels.extend([label] * 5)
    return augmented_data, augmented_labels

augmented_data, augmented_labels = augment_audio_data(all_audio_data, labels)
print(f"Artırılmış veri sayısı: {len(augmented_data)}")

from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(features_df.iloc[:, :-1], features_df['Label'])



Veri Dengeleme

In [ ]:
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(features_df.iloc[:, :-1], features_df['Label'])

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Etiketleri encode et ve one-hot dönüştür
label_encoder = LabelEncoder()
y_resampled_encoded = label_encoder.fit_transform(y_resampled)  # Etiketleri encode et
y_train = tf.keras.utils.to_categorical(label_encoder.transform(y_train), num_classes=len(label_encoder.classes_))
y_test = tf.keras.utils.to_categorical(label_encoder.transform(y_test), num_classes=len(label_encoder.classes_))


Model Oluşturma

In [ ]:
from tensorflow.keras import Input
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Dense(256, input_shape=(input_shape,), activation='relu'),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')  # Çıkış katmanı
    ])
    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return model

# Sınıf sayısını belirleme
num_classes = y_train.shape[1]  # One-hot encoded olduğu için sütun sayısı sınıf sayısını verir

# Model oluşturma
model = create_cnn_model(X_train.shape[1], num_classes)
model.summary()


Modeli Eğitme

In [ ]:
# Öğrenme oranı zamanlayıcısı
def scheduler(epoch, lr):
    return lr if epoch < 10 else float(lr * tf.math.exp(-0.1))

# Callbacks tanımlama
lr_scheduler = LearningRateScheduler(scheduler)
checkpointer = ModelCheckpoint(filepath='best_model.keras', save_best_only=True, monitor='val_loss', mode='min')

# Model eğitimi
history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=50,
    validation_data=(X_test, y_test),
    callbacks=[lr_scheduler, checkpointer]
)



Eğitim Geçmişini Görselleştirme

In [ ]:
def plot_history(history):
    plt.figure()
    plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
    plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
    plt.title('Doğruluk')
    plt.legend()
    plt.show()

    plt.figure()
    plt.plot(history.history['loss'], label='Eğitim Kaybı')
    plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
    plt.title('Kayıp')
    plt.legend()
    plt.show()

plot_history(history)


Yeni Ses Dosyasını Tahmin Etme

In [ ]:
# Gerekli kütüphanelerin yüklenmesi
!pip install torch torchvision torchaudio -q

import torchaudio
from IPython.display import Audio

# Ses dosyasını yükleme ve çalma
audio_path = '/content/drive/MyDrive/YSA/Test_submission/Test_submission/darbuka-drum-percussion-64018.wav'

# Torchaudio ile ses dosyasını yükle
audio, rate = torchaudio.load(audio_path)

# Ses bilgilerini ekrana yazdır
print(f"Ses Şekli: {audio.shape}, Örnekleme Oranı: {rate}")

# Ses dosyasını çal
print("Ses çalınıyor...")
Audio(audio.numpy().flatten(), rate=rate)


In [ ]:
# Yeni bir ses dosyasını tahmin etme
test_audio, test_sr = librosa.load('/content/drive/MyDrive/YSA/Test_submission/Test_submission/darbuka-drum-percussion-64018.wav', sr=22050)
test_features = extract_features(test_audio, test_sr).reshape(1, -1)

prediction = model.predict(test_features)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])
print(f"Tahmin Edilen Sınıf: {predicted_label[0]}")

# Test seti üzerinde modeli değerlendirme
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Seti Doğruluk: {test_accuracy:.4f}")
print(f"Test Seti Kayıp: {test_loss:.4f}")



In [ ]:
# Gerekli kütüphanelerin yüklenmesi
!pip install torch torchvision torchaudio -q

import torchaudio
from IPython.display import Audio

# Ses dosyasını yükleme ve çalma
audio_path = '/content/drive/MyDrive/YSA/Test_submission/Test_submission/ROOM_room6_MUS_pachelbel_DEV_lg.wav'

# Torchaudio ile ses dosyasını yükle
audio, rate = torchaudio.load(audio_path)

# Ses bilgilerini ekrana yazdır
print(f"Ses Şekli: {audio.shape}, Örnekleme Oranı: {rate}")

# Ses dosyasını çal
print("Ses çalınıyor...")
Audio(audio.numpy().flatten(), rate=rate)


In [ ]:
# Yeni bir ses dosyasını tahmin etme
test_audio, test_sr = librosa.load('/content/drive/MyDrive/YSA/Test_submission/Test_submission/ROOM_room6_MUS_pachelbel_DEV_lg.wav', sr=22050)
test_features = extract_features(test_audio, test_sr).reshape(1, -1)

prediction = model.predict(test_features)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])
print(f"Tahmin Edilen Sınıf: {predicted_label[0]}")

# Test seti üzerinde modeli değerlendirme
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Seti Doğruluk: {test_accuracy:.4f}")
print(f"Test Seti Kayıp: {test_loss:.4f}")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Test seti tahminleri
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Tahmin edilen sınıflar
y_test_classes = np.argmax(y_test, axis=1)  # Gerçek sınıflar

# Performans raporu
print("Sınıflandırma Raporu:")
print(classification_report(y_test_classes, y_pred_classes, target_names=label_encoder.classes_))

# Karışıklık matrisi
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
print("Karışıklık Matrisi:")
print(conf_matrix)
